In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))

In [ ]:
%run ./check_DOVS_METHODS.ipynb

In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re
import pickle
from pathlib import Path

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import copy
import itertools
import adjustText

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#-----
from DOVSAudit import DOVSAudit
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Utilities_dt
from Utilities_df import DFConstructType
import Plot_General
import Plot_Box_sns
import Plot_Hist
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer

# Analyze collected data

## AMI NonVee

In [ ]:
save_dir = r'C:\Users\s346557\Documents\LocalData\dovs_check\Mico2\NEWAF3'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
base_dirs = [
    r'C:\Users\s346557\Documents\LocalData\dovs_check\Mico2\AllOPCOs'
]
assert(all([os.path.exists(dir_i) for dir_i in base_dirs]))

In [ ]:
base_dirs

In [ ]:
outg_rec_nb_to_files_dicts = []
outg_rec_nb_to_files_ede_dicts = []
for base_dir in base_dirs:
    print(base_dir)
    base_dir_ami = os.path.join(base_dir, r'AMINonVee')
    base_dir_ede = os.path.join(base_dir, r'EndEvents')
    assert(os.path.exists(base_dir_ami))
    assert(os.path.exists(base_dir_ede))
    #--------------------------------------------------
    #--------------------------------------------------
    if os.path.exists(os.path.join(base_dir, 'outg_rec_nb_to_files_dict.pkl')):
        with open(os.path.join(base_dir, 'outg_rec_nb_to_files_dict.pkl'), 'rb') as handle:
            outg_rec_nb_to_files_dict = pickle.load(handle)
    else:
        #-------------------------
        paths = Utilities.find_all_paths(
            base_dir=base_dir_ami, 
            glob_pattern=r'ami_nonvee_[0-9]*.csv', 
            regex_pattern=None
        )
        paths=natsorted(paths)
        #-------------------------
        outg_rec_nbs_in_files = dict()
        for path in paths:
            assert(path not in outg_rec_nbs_in_files.keys())
            df = GenAn.read_df_from_csv(path)
            outg_rec_nbs_in_files[path] = df['OUTG_REC_NB_GPD_FOR_SQL'].unique().tolist()
        outg_rec_nb_to_files_dict = invert_file_to_outg_rec_nbs_dict(outg_rec_nbs_in_files)
        #-------------------------
        with open(os.path.join(base_dir, 'outg_rec_nb_to_files_dict.pkl'), 'wb') as handle:
            pickle.dump(outg_rec_nb_to_files_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    #-------------------------
    outg_rec_nb_to_files_dicts.append(outg_rec_nb_to_files_dict)
    #--------------------------------------------------
    #--------------------------------------------------
    if os.path.exists(os.path.join(base_dir, 'outg_rec_nb_to_files_ede_dict.pkl')):
        with open(os.path.join(base_dir, 'outg_rec_nb_to_files_ede_dict.pkl'), 'rb') as handle:
            outg_rec_nb_to_files_ede_dict = pickle.load(handle)
    else:
        #-------------------------
        paths_ede = Utilities.find_all_paths(
            base_dir=base_dir_ede, 
            glob_pattern=r'end_events_[0-9]*.csv', 
            regex_pattern=None
        )
        paths_ede=natsorted(paths_ede)
        #-------------------------
        outg_rec_nbs_in_files_ede = dict()
        for path in paths_ede:
            assert(path not in outg_rec_nbs_in_files_ede.keys())
            df = GenAn.read_df_from_csv(path)
            outg_rec_nbs_in_files_ede[path] = df['OUTG_REC_NB_GPD_FOR_SQL'].unique().tolist()
        outg_rec_nb_to_files_ede_dict = invert_file_to_outg_rec_nbs_dict(outg_rec_nbs_in_files_ede)
        #-------------------------
        with open(os.path.join(base_dir, 'outg_rec_nb_to_files_ede_dict.pkl'), 'wb') as handle:
            pickle.dump(outg_rec_nb_to_files_ede_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    #-------------------------
    outg_rec_nb_to_files_ede_dicts.append(outg_rec_nb_to_files_ede_dict)

In [ ]:
# There really shouldn't be any overlap in the outg_rec_nbs from each base_dir
# In the rare case this isn't desired, simply comment out
all_outg_rec_nbs = []
for dct_i in outg_rec_nb_to_files_dicts:
    for outg_rec_nb_i, files_i in dct_i.items():
        assert(outg_rec_nb_i not in all_outg_rec_nbs)
        all_outg_rec_nbs.append(outg_rec_nb_i)
#-------------------------
# Since there are no repeats, it is safe to simply combine all dicts
outg_rec_nb_to_files_dict = dict()
for dct_i in outg_rec_nb_to_files_dicts:
    outg_rec_nb_to_files_dict = outg_rec_nb_to_files_dict|dct_i
#-------------------------
assert(set(outg_rec_nb_to_files_dict.keys()).symmetric_difference(set(all_outg_rec_nbs))==set())

In [ ]:
# There really shouldn't be any overlap in the outg_rec_nbs from each base_dir
# In the rare case this isn't desired, simply comment out
all_outg_rec_nbs_ede = []
for dct_i in outg_rec_nb_to_files_ede_dicts:
    for outg_rec_nb_i, files_i in dct_i.items():
        assert(outg_rec_nb_i not in all_outg_rec_nbs_ede)
        all_outg_rec_nbs_ede.append(outg_rec_nb_i)
#-------------------------
# Since there are no repeats, it is safe to simply combine all dicts
outg_rec_nb_to_files_ede_dict = dict()
for dct_i in outg_rec_nb_to_files_ede_dicts:
    outg_rec_nb_to_files_ede_dict = outg_rec_nb_to_files_ede_dict|dct_i
#-------------------------
assert(set(outg_rec_nb_to_files_ede_dict.keys()).symmetric_difference(set(all_outg_rec_nbs_ede))==set())

# -----------------------------------------------------------------------------------------------------------

In [ ]:
mico2_df = pd.read_excel(r'C:\Users\s346557\Documents\LocalData\dovs_check\Mico2\Mico2_redOnly.xlsx', dtype=str)

In [ ]:
mico2_df=mico2_df.rename(columns={
    'Unnamed: 0':'outage_nb', 
    'Unnamed: 1':'dt_on_ts'
})
mico2_df=mico2_df[['outage_nb', 'dt_on_ts']]
mico2_df['dt_on_ts'] = pd.to_datetime(mico2_df['dt_on_ts'])
outage_nbs=mico2_df['outage_nb'].unique().tolist()

In [ ]:
# Need to get outg_rec_nbs
tmp_dovs = DOVSOutages(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args=None, 
    init_df_in_constructor=True,
    build_sql_function=DOVSOutages_SQL.build_sql_std_outage, 
    build_sql_function_kwargs=dict(
        outage_nbs=outage_nbs, 
        field_to_split='outage_nbs', 
        include_premise=True
    ), 
    build_consolidated=True
)
tmp_dovs_df = tmp_dovs.df.copy()


In [ ]:
# Outage numbers are re-used, which is annoying
assert(mico2_df['outage_nb'].nunique()==mico2_df.shape[0])
# tmp_dovs_df=tmp_dovs_df.groupby('OUTAGE_NB', as_index=False, group_keys=False).apply(
#     lambda x: x[x['DT_ON_TS']==mico2_df[mico2_df['outage_nb']==x.name]['dt_on_ts'].unique().tolist()[0]]
# )
tmp_dovs_df=tmp_dovs_df.groupby('OUTAGE_NB', as_index=False, group_keys=False).apply(
    lambda x: x[np.abs(x['DT_ON_TS']-mico2_df[mico2_df['outage_nb']==x.name]['dt_on_ts'].unique().tolist()[0])<pd.Timedelta('12H')]
)

In [ ]:
tmp_dovs_df

In [ ]:
outg_rec_nbs = tmp_dovs_df.index.get_level_values(0).unique().tolist()

In [ ]:
set(outg_rec_nbs).difference(set(all_outg_rec_nbs))

In [ ]:
print(f"# Missing = {len(set(outg_rec_nbs).difference(set(all_outg_rec_nbs)))}")

In [ ]:
outg_rec_nbs = [x for x in outg_rec_nbs if x in all_outg_rec_nbs]

In [ ]:
print(f'#outg_rec_nbs = {len(outg_rec_nbs)}')

# -----------------------------------------------------------------------------------------------------------

In [ ]:
include_suboutg_endpt_plots=True

calculate_by_PN = True
# combine_by_PN_likeness_thresh = pd.Timedelta('1 minutes')
combine_by_PN_likeness_thresh = pd.Timedelta('15 minutes')

expand_outg_search_time_tight = pd.Timedelta('1 hours')
expand_outg_search_time_loose = pd.Timedelta('12 hours')
use_est_outg_times=False
# use_est_outg_times=True
use_full_ede_outgs=False
run_outg_inclusion_assessment=True
max_pct_PNs_missing_allowed=0
# max_pct_PNs_missing_allowed=20
#-----
expand_outg_est_search_time = expand_outg_search_time_loose
if use_est_outg_times:
    expand_outg_search_time = expand_outg_search_time_tight
else:
    expand_outg_search_time = expand_outg_search_time_loose

In [ ]:
pdf_path = os.path.join(save_dir, r'Results.pdf')
pdf = PdfPages(pdf_path)
#-------------------------
pdf_path_dovs_beg = Utilities.append_to_path(
    pdf_path, 
    '_dovs_beg', 
    ext_to_find='.pdf', 
    append_to_end_if_ext_no_found=False
)
pdf_dovs_beg = PdfPages(pdf_path_dovs_beg)
#-------------------------
if include_suboutg_endpt_plots:
    pdf_path_2 = Utilities.append_to_path(
        pdf_path, 
        '_w_suboutg_endpt_plots', 
        ext_to_find='.pdf', 
        append_to_end_if_ext_no_found=False
    )
    pdf_2 = PdfPages(pdf_path_2)
#-------------------------
pdf_path_n_w_power = os.path.join(save_dir, r'n_w_power_v_time.pdf')
pdf_n_w_power = PdfPages(pdf_path_n_w_power)
#-----
pdf_path_n_w_power_dovs_beg = Utilities.append_to_path(
    pdf_path_n_w_power, 
    '_dovs_beg', 
    ext_to_find='.pdf', 
    append_to_end_if_ext_no_found=False
)
pdf_n_w_power_dovs_beg = PdfPages(pdf_path_n_w_power_dovs_beg) 

In [ ]:
n_PNs_w_power_threshold = 95

In [ ]:
fig_num=0
all_detailed_summary_dfs=[]
all_detailed_summary_dfs_dovs_beg=[]
ci_cmi_summary_df = pd.DataFrame(columns=[
    'outg_rec_nb', 
    'ci_dovs',  'ci_ami', 'ci_ami_dovs_beg', 
    'cmi_dovs', 'cmi_ami', 'cmi_ami_dovs_beg'
])
warnings_text = ''

#-------------------------
# Build dovs_df
dovs = DOVSOutages(
    df_construct_type         = DFConstructType.kRunSqlQuery, 
    contstruct_df_args        = None, 
    init_df_in_constructor    = True,
    build_sql_function        = DOVSOutages_SQL.build_sql_std_outage, 
    build_sql_function_kwargs = dict(
        outg_rec_nbs=outg_rec_nbs, 
        field_to_split='outg_rec_nbs', 
        include_premise=True
    ), 
    build_consolidated        = True
)
dovs_df = dovs.df.copy()

#-------------------------
# Now, iterate through all outages
for i_outg, outg_rec_nb in enumerate(outg_rec_nbs):
    print(f'\n\ti_outg: {i_outg+1}/{len(outg_rec_nbs)}')
    print(f'\toutg_rec_nb = {outg_rec_nb}')
    #--------------------------------------------------
    audit_i = DOVSAudit(
        outg_rec_nb=outg_rec_nb
    )
    audit_i.load_ami_from_csvs(
        paths                          = outg_rec_nb_to_files_dict[outg_rec_nb], 
        slicers                        = None, 
        ami_df_info_dict               = None, 
        run_std_init                   = True, 
        cols_and_types_to_convert_dict = None, 
        to_numeric_errors              = 'coerce', 
        drop_na_rows_when_exception    = True, 
        drop_unnamed0_col              = True, 
        pd_read_csv_kwargs             = None, 
        make_all_columns_lowercase     = False, 
        assert_all_cols_equal          = True, 
        min_fsize_MB                   = None
    )
    #--------------------------------------------------
    if audit_i.ami_df_i.shape[0]==0:
        continue

    #-------------------------
    # Need to load dovs before running self assessment below
    audit_i.load_dovs(
        dovs_df           = dovs_df, 
        dovs_df_info_dict = None
    )
    
    if run_outg_inclusion_assessment:
        to_include_i = audit_i.self_assess_outage_inclusion_requirements(max_pct_PNs_missing_allowed, None)
        if not to_include_i:
            print(f'outg_rec_nb={outg_rec_nb} did not pass inclusion requirements, skipping!!!!!')
            continue
    #-------------------------    
    n_SNs  = audit_i.ami_df_i['serialnumber'].nunique()
    n_PNs  = audit_i.ami_df_i['aep_premise_nb'].nunique()
    
    #----------------------------------------------------------------------------------------------------
    # NOTE: Can save time by grabbing ede_df_i then performing tz conversion and adding DOVS
    if outg_rec_nb not in outg_rec_nb_to_files_ede_dict.keys():
        ede_df_i=None
    else:
        audit_i.load_ede_from_csvs(
            paths                          = outg_rec_nb_to_files_ede_dict[outg_rec_nb], 
            slicers                        = None, 
            ede_df_info_dict               = None, 
            run_std_init                   = True, 
            cols_and_types_to_convert_dict = None, 
            to_numeric_errors              = 'coerce', 
            drop_na_rows_when_exception    = True, 
            drop_unnamed0_col              = True, 
            pd_read_csv_kwargs             = None, 
            make_all_columns_lowercase     = False, 
            assert_all_cols_equal          = True, 
            min_fsize_MB                   = None
        )
    #----------------------------------------------------------------------------------------------------
    audit_i.build_best_ests_df()
    #-------------------------
    # Get the outage time from DOVS
    dovs_outg_t_beg_end              = audit_i.dovs_outg_t_beg_end
    dovs_outg_t_beg, dovs_outg_t_end = dovs_outg_t_beg_end
    #-------------------------
    # Get the CI and CMI from DOVS
    ci_cmi_dovs       = audit_i.ci_cmi_dovs
    ci_dovs, cmi_dovs = ci_cmi_dovs
    #-------------------------
    # Get the number of premises from DOVS
    n_PNs_dovs = audit_i.n_PNs_dovs
    #-------------------------
    # Get the outage number from DOVS
    outage_nb = audit_i.outage_nb
    
    #--------------------------------------------------
    # cnsrvtv_out_t_beg/_end are used for placing bounds on the plots generated
    # If audit_i.best_ests_df has non-zero size (meaning the algorithm found outages), use to set plotting time.
    # Otherwise, use dovs_outg_t_beg/_end
    # NOTE: If one did not want to show any data which was thrown out due to overlapping, one would want to
    #         update cnsrvtv_out_t_beg/_end after the identify_dovs_overlaps_from_best_ests procedure below 
    #         (and subsequent trimming of audit_i.best_ests_df)
    if audit_i.best_ests_df.shape[0]>0:
        cnsrvtv_out_t_beg = np.min([audit_i.best_ests_df['conservative_min'].min(), dovs_outg_t_beg])
        cnsrvtv_out_t_end = np.max([audit_i.best_ests_df['conservative_max'].max(), dovs_outg_t_end])
    else:
        cnsrvtv_out_t_beg = dovs_outg_t_beg
        cnsrvtv_out_t_end = dovs_outg_t_end
    #--------------------------------------------------
    if audit_i.best_ests_df.shape[0]>0:
        # Identify and handle any overlaps with other DOVS events
        #-------------------------
        # dovs_sql_fcn=DOVSOutages_SQL.build_sql_std_outage
        dovs_sql_fcn=DOVSOutages_SQL.build_sql_outage
        audit_i.identify_overlaps(overlaps_dovs_sql_fcn = dovs_sql_fcn)
        #-------------------------
        # If any PN has one or more overlapping DOVS events, output info to file
        n_PNs_w_overlap = (audit_i.overlap_outgs_for_PNs_df['n_overlap']>0).sum()
        if n_PNs_w_overlap>0:
            print(f'Need to output to new file\n\tn_PNs_w_overlap={n_PNs_w_overlap}')
        #-------------------------
        # If any PN which lost power has one or more overlapping DOVS events, stop analysis
        n_out_PNs_w_overlap = audit_i.overlap_outgs_for_PNs_df[audit_i.overlap_outgs_for_PNs_df['lost_power']==True]['overlap_outg_rec_nbs'].apply(lambda x: len(x)>0).sum()
        if n_out_PNs_w_overlap>0:
            print('STOP analysis')
            continue
        #-------------------------
        audit_i.resolve_overlapping_audits()
        #-----
        ci_ami  = audit_i.ci
        cmi_ami = audit_i.cmi            
        #-------------------------
        ami_df_i = audit_i.ami_df_i.copy()
        # In ami_df_i, mark any entries which were essentially removed via the identify_dovs_overlaps_from_best_ests
        #   and removal procedure above
        ami_df_i = DOVSAudit.set_removed_due_to_overlap_in_ami_df_i(
            ami_df_i                   = ami_df_i, 
            best_ests_df               = audit_i.best_ests_df_w_keep_info.copy(), 
            PN_col                     = 'aep_premise_nb', 
            time_idfr                  = 'starttimeperiod_local', 
            PN_col_be                  = 'PN', 
            keep_col_be                = 'keep', 
            overlap_times_col_be       = 'overlap_times', 
            removed_due_to_overlap_col = 'removed_due_to_overlap'
        )
    else:
        ci_ami  = 0
        cmi_ami = 0
        ami_df_i = audit_i.ami_df_i.copy()
    #--------------------------------------------------
    if audit_i.best_ests_df.shape[0]>0:
        best_ests_df_dovs_beg = DOVSAudit.alter_best_ests_df_using_dovs_outg_t_beg(
            best_ests_df = audit_i.best_ests_df,
            dovs_df      = dovs_df, 
            outg_rec_nb  = outg_rec_nb
        )
        if calculate_by_PN:
            ci_ami_dovs_beg  = best_ests_df_dovs_beg['PN'].nunique()
        else:
            ci_ami_dovs_beg  = best_ests_df_dovs_beg['SN'].nunique()
        cmi_ami_dovs_beg = (best_ests_df_dovs_beg['winner_max']-best_ests_df_dovs_beg['winner_min']).sum().total_seconds()/60
    else:
        best_ests_df_dovs_beg = audit_i.best_ests_df.copy()
        ci_ami_dovs_beg  = ci_ami
        cmi_ami_dovs_beg = cmi_ami        
    #--------------------------------------------------
    dovs_df_i = DOVSOutages.retrieve_outage_from_dovs_df(
        dovs_df                  = dovs_df, 
        outg_rec_nb              = audit_i.outg_rec_nb, 
        outg_rec_nb_idfr         = 'index', 
        assert_outg_rec_nb_found = True
    )    
    #--------------------------------------------------
    if audit_i.best_ests_df.shape[0]>0:
        means_df, best_ests_df_w_db_lbl = DOVSAudit.get_mean_times_w_dbscan(
            best_ests_df                  = audit_i.best_ests_df, 
            eps_min                       = 5, 
            min_samples                   = 2, 
            ests_to_include_in_clustering = ['winner_min', 'winner_max'],
            ests_to_include_in_output     = [
                'winner_min', 'winner_max', 
                'conservative_min', 'conservative_max', 
                'zero_times_min', 'zero_times_max'
            ], 
            return_labelled_best_ests_df  = True
        )
        #-------------------------
        n_PNs_w_power_srs = DOVSAudit.build_n_PNs_w_power_srs(
            best_ests_df  = audit_i.best_ests_df, 
            ami_df_i      = ami_df_i, 
            return_pct    = True, 
            PN_col        = 'PN', 
            t_min_col     = 'winner_min', 
            t_max_col     = 'winner_max', 
            i_outg_col    = 'i_outg', 
            PN_col_ami_df = 'aep_premise_nb'
        )        
        #-------------------------
        detailed_summary_df_i = DOVSAudit.build_detailed_summary_df(
            means_df              = means_df, 
            best_ests_df_w_db_lbl = best_ests_df_w_db_lbl,
            CI_tot                = ci_ami, 
            CMI_tot               = cmi_ami, 
            n_PNs_ami             = n_PNs,
            outg_rec_nb           = outg_rec_nb, 
            dovs_df_i             = dovs_df_i, 
            warnings_flag         = audit_i.warnings_flag, 
            db_label_col          = 'db_label', 
            winner_min_col        = 'winner_min', 
            winner_max_col        = 'winner_max', 
            PN_col                = 'PN' if calculate_by_PN else 'SN', 
            i_outg_col            = 'i_outg'
        )
        #----------
        detailed_summary_df_i[f'first_above_thresh ({n_PNs_w_power_threshold})'] = None
        detailed_summary_df_i[f'last_above_thresh ({n_PNs_w_power_threshold})']  = None
        frst_abv, last_abv = get_first_last_above_threshold(
            n_PNs_w_power_srs = n_PNs_w_power_srs, 
            threshold         = n_PNs_w_power_threshold
        )
        #-----
        detailed_summary_df_i.iloc[
            0, 
            detailed_summary_df_i.columns.tolist().index(f'first_above_thresh ({n_PNs_w_power_threshold})')
        ] = frst_abv
        #-----
        detailed_summary_df_i.iloc[
            0, 
            detailed_summary_df_i.columns.tolist().index(f'last_above_thresh ({n_PNs_w_power_threshold})')
        ] = last_abv        
        #-------------------------
        all_detailed_summary_dfs.append(detailed_summary_df_i)
        
        #-------------------------
        warnings_text += audit_i.generate_warnings_text()
    else:
        means_df, best_ests_df_w_db_lbl = None, None
        n_PNs_w_power_srs = None
    #--------------------------------------------------
    if best_ests_df_dovs_beg.shape[0]>0:
        means_df_dovs_beg, best_ests_df_dovs_beg_w_db_lbl = DOVSAudit.get_mean_times_w_dbscan(
            best_ests_df                  = best_ests_df_dovs_beg, 
            eps_min                       = 5, 
            min_samples                   = 2, 
            ests_to_include_in_clustering = ['winner_min', 'winner_max'],
            ests_to_include_in_output     = [
                'winner_min', 'winner_max', 
                'conservative_min', 'conservative_max', 
                'zero_times_min', 'zero_times_max'
            ], 
            return_labelled_best_ests_df  = True
        )
        #-------------------------
        n_PNs_w_power_srs_dovs_beg = DOVSAudit.build_n_PNs_w_power_srs(
            best_ests_df  = best_ests_df_dovs_beg, 
            ami_df_i      = ami_df_i, 
            return_pct    = True, 
            PN_col        = 'PN', 
            t_min_col     = 'winner_min', 
            t_max_col     = 'winner_max', 
            i_outg_col    = 'i_outg', 
            PN_col_ami_df = 'aep_premise_nb'
        )        
        #-------------------------
        detailed_summary_df_dovs_beg_i = DOVSAudit.build_detailed_summary_df(
            means_df              = means_df_dovs_beg, 
            best_ests_df_w_db_lbl = best_ests_df_dovs_beg_w_db_lbl,
            CI_tot                = ci_ami_dovs_beg, 
            CMI_tot               = cmi_ami_dovs_beg, 
            n_PNs_ami             = n_PNs,
            outg_rec_nb           = outg_rec_nb, 
            dovs_df_i             = dovs_df_i, 
            warnings_flag         = audit_i.warnings_flag, 
            db_label_col          = 'db_label', 
            winner_min_col        = 'winner_min', 
            winner_max_col        = 'winner_max', 
            PN_col                = 'PN' if calculate_by_PN else 'SN', 
            i_outg_col            = 'i_outg'
        )
        #----------
        detailed_summary_df_dovs_beg_i[f'first_above_thresh ({n_PNs_w_power_threshold})'] = None
        detailed_summary_df_dovs_beg_i[f'last_above_thresh ({n_PNs_w_power_threshold})']  = None
        frst_abv, last_abv = get_first_last_above_threshold(
            n_PNs_w_power_srs = n_PNs_w_power_srs_dovs_beg, 
            threshold         = n_PNs_w_power_threshold
        )
        #-----
        detailed_summary_df_dovs_beg_i.iloc[
            0, 
            detailed_summary_df_dovs_beg_i.columns.tolist().index(f'first_above_thresh ({n_PNs_w_power_threshold})')
        ] = frst_abv
        #-----
        detailed_summary_df_dovs_beg_i.iloc[
            0, 
            detailed_summary_df_dovs_beg_i.columns.tolist().index(f'last_above_thresh ({n_PNs_w_power_threshold})')
        ] = last_abv        
        #-------------------------
        all_detailed_summary_dfs_dovs_beg.append(detailed_summary_df_dovs_beg_i)
    else:
        means_df_dovs_beg, best_ests_df_dovs_beg_w_db_lbl = None, None
    #-------------------------
    ci_cmi_summary_df = pd.concat([
        ci_cmi_summary_df, 
        pd.DataFrame(
            dict(
                outg_rec_nb=outg_rec_nb, 
                ci_dovs=ci_dovs,   ci_ami=ci_ami, ci_ami_dovs_beg=ci_ami_dovs_beg, 
                cmi_dovs=cmi_dovs, cmi_ami=cmi_ami, cmi_ami_dovs_beg=cmi_ami_dovs_beg
            ), 
            index=[ci_cmi_summary_df.shape[0]]
        )
    ])
    #----------------------------------------------------------------------------------------------------
    # ######################### PLOTTING #########################
    #----------------------------------------------------------------------------------------------------
    #--------------------------------------------------
    # Instead of using get_full_part_not_outage_subset_dfs, simply grab the PNs which suffered
    #   outages from best_ests_df
    if audit_i.best_ests_df.shape[0]>0:
        outg_SNs = audit_i.best_ests_df['PN'].unique().tolist()
        removed_due_to_overlap_col = 'removed_due_to_overlap'
    else:
        outg_SNs = []
        removed_due_to_overlap_col = None
    #-----
    ami_df_i_out      = ami_df_i[ami_df_i['aep_premise_nb'].isin(outg_SNs)]
    ami_df_i_not_out  = ami_df_i[~ami_df_i['aep_premise_nb'].isin(outg_SNs)]  
    
    #--------------------------------------------------
    if audit_i.best_ests_df_w_keep_info is not None and audit_i.best_ests_df_w_keep_info.shape[0]>0:
        ptntl_ovrlp_outg_rec_nbs = list(set(audit_i.best_ests_df_w_keep_info['overlap_DOVS'].sum()))
        if len(ptntl_ovrlp_outg_rec_nbs)>0:
            ovrlp_dovs = DOVSOutages(
                df_construct_type=DFConstructType.kRunSqlQuery, 
                contstruct_df_args=None, 
                init_df_in_constructor=True,
                build_sql_function=DOVSOutages_SQL.build_sql_outage, 
                build_sql_function_kwargs=dict(
                    outg_rec_nbs=ptntl_ovrlp_outg_rec_nbs, 
                    include_premise=True
                ), 
                build_consolidated=True
            )
            other_dovs_events_df = ovrlp_dovs.df.reset_index().copy()
        else:
            other_dovs_events_df = None
    else:
        other_dovs_events_df = None

    #----------------------------------------------------------------------------------------------------
    fig, axs = plot_all_out_not_NEW(
        fig_num                    = fig_num, 
        ami_df_i                   = ami_df_i, 
        ami_df_i_out               = ami_df_i_out, 
        ami_df_i_not_out           = ami_df_i_not_out, 
        dovs_outg_t_beg            = dovs_outg_t_beg, 
        dovs_outg_t_end            = dovs_outg_t_end, 
        cnsrvtv_out_t_beg          = cnsrvtv_out_t_beg, 
        cnsrvtv_out_t_end          = cnsrvtv_out_t_end, 
        means_df                   = means_df, 
        outg_rec_nb                = outg_rec_nb, 
        outage_nb                  = outage_nb, 
        n_PNs_dovs                 = n_PNs_dovs, 
        ci_dovs                    = ci_dovs, 
        cmi_dovs                   = cmi_dovs, 
        ci_ami                     = ci_ami, 
        cmi_ami                    = cmi_ami, 
        name                       = 'AMI', 
        results_2_dict             = dict(
            ci_ami   = ci_ami_dovs_beg, 
            cmi_ami  = cmi_ami_dovs_beg, 
            means_df = means_df_dovs_beg, 
            name = 'AMI w/ DOVS t_beg'
        ), 
        expand_time                = pd.Timedelta('1 hour'), 
        removed_due_to_overlap_col = removed_due_to_overlap_col, 
        mean_keys_to_include       = ['winner', 'conservative', 'zero_times'], 
        default_subplots_args      = dict(n_x=2, n_y=2, row_major=True, sharex=True), 
        other_dovs_events_df       = other_dovs_events_df, 
        leg_i_plot                 = 1, 
        leg_kwargs                 = dict(ncols=1, fontsize=15, bbox_to_anchor=(1, 1.2)), 
        ci_info_fontsize           = 16, 
        left_text_x                = 0.915  
    )

    if n_PNs_w_power_srs is not None:
        fig, axs[3] = DOVSAudit.plot_n_PNs_w_power_srs(
            n_PNs_w_power_srs = n_PNs_w_power_srs, 
            simp_freq         = '1T', 
            threshold         = n_PNs_w_power_threshold, 
            fig_num           = fig_num, 
            fig_ax            = (fig, axs[3]), 
            threshold_color   = 'magenta'
        )
        
    for ax_i in axs:
        ax_i.xaxis.set_tick_params(labelbottom=True)

    fig_num += 1
    pdf.savefig(fig, bbox_inches='tight')
    if include_suboutg_endpt_plots:
        pdf_2.savefig(fig, bbox_inches='tight')
    plt.close(fig)
    
    #----------------------------------------------------------------------------------------------------
    if means_df_dovs_beg is not None:
        fig, axs = plot_all_out_not_NEW(
            fig_num              = fig_num, 
            ami_df_i             = ami_df_i, 
            ami_df_i_out         = ami_df_i_out, 
            ami_df_i_not_out     = ami_df_i_not_out, 
            dovs_outg_t_beg      = dovs_outg_t_beg, 
            dovs_outg_t_end      = dovs_outg_t_end, 
            cnsrvtv_out_t_beg    = cnsrvtv_out_t_beg, 
            cnsrvtv_out_t_end    = cnsrvtv_out_t_end, 
            means_df             = means_df_dovs_beg, 
            outg_rec_nb          = outg_rec_nb, 
            outage_nb            = outage_nb, 
            n_PNs_dovs           = n_PNs_dovs, 
            ci_dovs              = ci_dovs, 
            cmi_dovs             = cmi_dovs, 
            ci_ami               = ci_ami_dovs_beg, 
            cmi_ami              = cmi_ami_dovs_beg, 
            name                 = 'AMI w/ DOVS t_beg', 
            results_2_dict       = dict(
                ci_ami   = ci_ami, 
                cmi_ami  = cmi_ami, 
                means_df = means_df, 
                name = 'AMI'
            ), 
            expand_time          = pd.Timedelta('1 hour'), 
            removed_due_to_overlap_col = removed_due_to_overlap_col, 
            mean_keys_to_include = ['winner', 'conservative', 'zero_times'], 
            default_subplots_args      = dict(n_x=2, n_y=2, row_major=True, sharex=True), 
            other_dovs_events_df       = other_dovs_events_df, 
            leg_i_plot                 = 1, 
            leg_kwargs                 = dict(ncols=1, fontsize=15, bbox_to_anchor=(1, 1.2)), 
            ci_info_fontsize           = 16, 
            left_text_x                = 0.915  
        )
        
        if n_PNs_w_power_srs_dovs_beg is not None:
            fig, axs[3] = DOVSAudit.plot_n_PNs_w_power_srs(
                n_PNs_w_power_srs = n_PNs_w_power_srs_dovs_beg, 
                simp_freq         = '1T', 
                threshold         = n_PNs_w_power_threshold, 
                fig_num           = fig_num, 
                fig_ax            = (fig, axs[3]), 
                threshold_color   = 'magenta'
            )

        for ax_i in axs:
            ax_i.xaxis.set_tick_params(labelbottom=True)

        fig_num += 1
        pdf_dovs_beg.savefig(fig, bbox_inches='tight')
        plt.close(fig)
    
    #----------------------------------------------------------------------------------------------------
    if means_df is not None and include_suboutg_endpt_plots:
        fig, axs = plot_suboutg_endpts(
            fig_num               = fig_num, 
            ami_df_i              = ami_df_i, 
            means_df              = means_df, 
            best_ests_df_w_db_lbl = best_ests_df_w_db_lbl, 
            dovs_outg_t_beg       = dovs_outg_t_beg, 
            dovs_outg_t_end       = dovs_outg_t_end, 
            outg_rec_nb           = outg_rec_nb, 
            expand_time           = pd.Timedelta('15 minutes'), 
            mean_keys_to_include  = ['winner', 'conservative', 'zero_times']
        )
        #-------------------------
        fig_num += 1
        pdf_2.savefig(fig, bbox_inches='tight')     
        plt.close(fig)
        
    #----------------------------------------------------------------------------------------------------
    if audit_i.best_ests_df.shape[0]>0:
        _, fig, ax = DOVSAudit.build_n_PNs_w_power_srs_and_plot(
            best_ests_df  = audit_i.best_ests_df, 
            ami_df_i      = ami_df_i, 
            return_pct    = True, 
            simp_freq     = '1T', 
            threshold     = n_PNs_w_power_threshold, 
            fig_num       = fig_num, 
            title         = f"OUTG_REC_NB = {outg_rec_nb}", 
            PN_col        = 'PN', 
            t_min_col     = 'winner_min', 
            t_max_col     = 'winner_max', 
            i_outg_col    = 'i_outg', 
            PN_col_ami_df = 'aep_premise_nb'
        )
        #-------------------------
        fig_num += 1
        pdf_n_w_power.savefig(fig, bbox_inches='tight')     
        plt.close(fig)
    #----------------------------------------------------------------------------------------------------
    if best_ests_df_dovs_beg.shape[0]>0:
        _, fig, ax = DOVSAudit.build_n_PNs_w_power_srs_and_plot(
            best_ests_df  = best_ests_df_dovs_beg, 
            ami_df_i      = ami_df_i, 
            return_pct    = True, 
            simp_freq     = '1T', 
            threshold     = n_PNs_w_power_threshold, 
            fig_num       = fig_num, 
            title         = f"OUTG_REC_NB = {outg_rec_nb}", 
            PN_col        = 'PN', 
            t_min_col     = 'winner_min', 
            t_max_col     = 'winner_max', 
            i_outg_col    = 'i_outg', 
            PN_col_ami_df = 'aep_premise_nb'
        )
        #-------------------------
        fig_num += 1
        pdf_n_w_power_dovs_beg.savefig(fig, bbox_inches='tight')     
        plt.close(fig)
        
#----------------------------------------------------------------------------------------------------
detailed_summary_df          = pd.concat(all_detailed_summary_dfs)
detailed_summary_df_dovs_beg = pd.concat(all_detailed_summary_dfs_dovs_beg)

In [ ]:
pdf.close()
pdf_dovs_beg.close()
if include_suboutg_endpt_plots:
    pdf_2.close()
pdf_n_w_power.close()
pdf_n_w_power_dovs_beg.close()

In [ ]:
ci_cmi_summary_df['ci_dovs']         = ci_cmi_summary_df['ci_dovs'].astype(float)
ci_cmi_summary_df['ci_ami']          = ci_cmi_summary_df['ci_ami'].astype(float)
ci_cmi_summary_df['ci_ami_dovs_beg'] = ci_cmi_summary_df['ci_ami_dovs_beg'].astype(float)
#-----
ci_cmi_summary_df['delta_ci_dovs_ami']  = ci_cmi_summary_df['ci_dovs']-ci_cmi_summary_df['ci_ami']
ci_cmi_summary_df['delta_cmi_dovs_ami'] = ci_cmi_summary_df['cmi_dovs']-ci_cmi_summary_df['cmi_ami']
#-----
ci_cmi_summary_df['delta_ci_dovs_ami_dovs_beg']  = ci_cmi_summary_df['ci_dovs']-ci_cmi_summary_df['ci_ami_dovs_beg']
ci_cmi_summary_df['delta_cmi_dovs_ami_dovs_beg'] = ci_cmi_summary_df['cmi_dovs']-ci_cmi_summary_df['cmi_ami_dovs_beg']
#-----
# For plotting purposes, make a outg_rec_in column which is simply 0 to delta_df.shape[0]-1
ci_cmi_summary_df['outg_rec_int'] = range(ci_cmi_summary_df.shape[0])
#-----

In [ ]:
detailed_summary_df.to_pickle(os.path.join(save_dir, r'detailed_summary.pkl'))
detailed_summary_df_dovs_beg.to_pickle(os.path.join(save_dir, r'detailed_summary_dovs_beg.pkl'))
ci_cmi_summary_df.to_pickle(os.path.join(save_dir, r'ci_cmi_summary.pkl'))
#-----
detailed_summary_df.to_csv(os.path.join(save_dir, r'detailed_summary.csv'))
detailed_summary_df_dovs_beg.to_csv(os.path.join(save_dir, r'detailed_summary_dovs_beg.csv'))
ci_cmi_summary_df.to_csv(os.path.join(save_dir, r'ci_cmi_summary.csv'))
#-----
with open(os.path.join(save_dir, r'warnings.txt'), 'w') as f:
    f.write(warnings_text)

In [ ]:
assert(0)

# ===========================================================

In [ ]:
dovs_df

In [ ]:
dovs_df_i

In [ ]:
dovs = DOVSOutages(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args=None, 
    init_df_in_constructor=True,
    build_sql_function=DOVSOutages_SQL.build_sql_outage, 
    build_sql_function_kwargs=dict(
        outg_rec_nbs=outg_rec_nbs, 
        field_to_split='outg_rec_nbs', 
        include_DOVS_PREMISE_DIM=True, 
        include_DOVS_MASTER_GEO_DIM=True, 
        include_DOVS_OUTAGE_ATTRIBUTES_DIM=True, 
        include_DOVS_CLEARING_DEVICE_DIM=True, 
        include_DOVS_EQUIPMENT_TYPES_DIM=True, 
        include_DOVS_OUTAGE_CAUSE_TYPES_DIM=True
    ), 
    build_consolidated=True
)
dovs_df_0 = dovs.df.copy()

In [ ]:
dovs = DOVSOutages(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args=None, 
    init_df_in_constructor=True,
    build_sql_function=DOVSOutages_SQL.build_sql_outage, 
    build_sql_function_kwargs=dict(
        outg_rec_nbs=outg_rec_nbs, 
        field_to_split='outg_rec_nbs', 
        include_DOVS_PREMISE_DIM=True, 
        include_DOVS_MASTER_GEO_DIM=True, 
        include_DOVS_OUTAGE_ATTRIBUTES_DIM=True, 
        include_DOVS_CLEARING_DEVICE_DIM=True, 
        include_DOVS_EQUIPMENT_TYPES_DIM=True, 
        include_DOVS_OUTAGE_CAUSE_TYPES_DIM=True
    ), 
    build_consolidated=False
)
dovs_df = dovs.df.copy()
dovs_df = dovs_df.set_index('OUTG_REC_NB')

In [ ]:
dovs_df

In [ ]:
dovs_df_info_dict = dict(
    is_consolidated  = True,
    outg_rec_nb_idfr = 'index', 
    PN_col           = 'PREMISE_NB', 
    PNs_col          = 'premise_nbs', 
    outg_t_beg_col   = 'DT_OFF_TS_FULL', 
    outg_t_end_col   = 'DT_ON_TS'
)

In [ ]:
outg_rec_nb_idfr_loc = Utilities_df.get_idfr_loc(
    df = dovs_df, 
    idfr = dovs_df_info_dict['outg_rec_nb_idfr']
    
)

In [ ]:
outg_rec_nb_idfr_loc

In [ ]:
# If the outg_rec_nbs are in the index, then reset_index must be called for
#   consolidate_df_outage to run properly
if outg_rec_nb_idfr_loc[1]:
    outg_rec_nb_idx_lvl = outg_rec_nb_idfr_loc[0]
    #-----
    if dovs_df.index.names[outg_rec_nb_idx_lvl]:
        outg_rec_nb_col = dovs_df.index.names[outg_rec_nb_idx_lvl]
    else:
        outg_rec_nb_col = 'OUTG_REC_NB_'+Utilities.generate_random_string(str_len=4)
        assert(outg_rec_nb_col not in dovs_df.columns.tolist())
        assert(outg_rec_nb_col not in list(dovs_df.index.names))
        dovs_df.index = dovs_df.index.set_names(outg_rec_nb_col, level=outg_rec_nb_idx_lvl)
    #-----
    # Set the outg_rec_nb_col and drop index
    dovs_df[outg_rec_nb_col] = dovs_df.index.get_level_values(outg_rec_nb_idx_lvl)
    if dovs_df.index.nlevels==1:
        # NOTE: Values already placed in outg_rec_nb_col above, hence why drop=True below
        dovs_df = dovs_df.reset_index(drop=True)
    else:
        dovs_df = dovs_df.droplevel(outg_rec_nb_idx_lvl, axis=0)
    #-----
    assert(outg_rec_nb_col in dovs_df.columns.tolist())
else:
    outg_rec_nb_col = outg_rec_nb_idfr_loc[0]

In [ ]:
# OFF_TM and REST_TM are premise specific, and should not be included
cols_to_drop = ['OFF_TM', 'REST_TM']
cols_to_drop = list(set(cols_to_drop).intersection(set(dovs_df.columns.tolist())))
#-----
# PN_col must be in dovs_df
assert(dovs_df_info_dict['PN_col'] in dovs_df.columns.tolist())
#-----
# All columns exepct premise numbers (and cols_to_drop) should be shared by groups
cols_shared_by_group = [x for x in dovs_df.columns.tolist() 
                        if x not in [outg_rec_nb_col, dovs_df_info_dict['PN_col']]+cols_to_drop]

In [ ]:
dovs_df = DOVSOutages.consolidate_df_outage(
    df_outage                    = dovs_df, 
    outg_rec_nb_col              = outg_rec_nb_col, 
    addtnl_grpby_cols            = None, 
    cols_shared_by_group         = cols_shared_by_group, 
    cols_to_collect_in_lists     = [dovs_df_info_dict['PN_col']], 
    allow_duplicates_in_lists    = False, 
    allow_NaNs_in_lists          = False, 
    recover_uniqueness_violators = True, 
    gpby_dropna                  = False, 
    rename_cols                  = None,     
    premise_nb_col               = dovs_df_info_dict['PN_col'], 
    premise_nbs_col              = dovs_df_info_dict['PNs_col'], 
    cols_to_drop                 = cols_to_drop, 
    sort_PNs                     = True, 
    drop_null_premise_nbs        = True, 
    set_outg_rec_nb_as_index     = True,
    drop_outg_rec_nb_if_index    = True, 
    verbose                      = False
)

In [ ]:
dovs_df_0.sort_index().equals(dovs_df.sort_index())

In [ ]:
to_include_i = assess_outage_inclusion_requirements(
    ami_df_i=ami_df_i, 
    outg_rec_nb=outg_rec_nb, 
    dovs_df=dovs_df, 
    max_pct_PNs_missing_allowed=max_pct_PNs_missing_allowed, 
    ami_df_i_info_dict=None, 
    dovs_df_info_dict=None, 
    check_found_ami_for_all_SNs_kwargs=None

)
print(f"to_include_i==audit_i?: {to_include_i==audit_i.self_assess_outage_inclusion_requirements(max_pct_PNs_missing_allowed, None)}")